In [124]:
'''
https://www.kaggle.com/code/allegich/lumbar-rsna-2024-visualizing-eda-sub/notebook
'''

'\nhttps://www.kaggle.com/code/allegich/lumbar-rsna-2024-visualizing-eda-sub/notebook\n'

In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 필요한 만큼만 메모리를 사용하도록 설정
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
        # 특정 GPU에 연산을 할당
        tf.config.set_visible_devices(gpus[0], 'GPU')
        
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        
    except RuntimeError as e:
        print(e)


1 Physical GPUs, 1 Logical GPU


In [3]:
import numpy as np
import pandas as pd

import os
import time

# plots
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import plotly.express as px
import plotly.express as px
import cv2

import pydicom as dicom # dicom
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import warnings # warning handling
warnings.filterwarnings('ignore')
import glob
import json
import collections

In [4]:
defaul_color_1='pink'

In [5]:
path = 'spine/'

df_train_main  = pd.read_csv(path + 'train.csv')
df_train_label = pd.read_csv(path + 'train_label_coordinates.csv')
df_train_desc  = pd.read_csv(path + 'train_series_descriptions.csv')
df_test_desc   = pd.read_csv(path + 'test_series_descriptions.csv')
df_sub         = pd.read_csv(path + 'sample_submission.csv')

In [6]:
df_train_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1975 entries, 0 to 1974
Data columns (total 26 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   study_id                                1975 non-null   int64 
 1   spinal_canal_stenosis_l1_l2             1974 non-null   object
 2   spinal_canal_stenosis_l2_l3             1974 non-null   object
 3   spinal_canal_stenosis_l3_l4             1974 non-null   object
 4   spinal_canal_stenosis_l4_l5             1974 non-null   object
 5   spinal_canal_stenosis_l5_s1             1974 non-null   object
 6   left_neural_foraminal_narrowing_l1_l2   1973 non-null   object
 7   left_neural_foraminal_narrowing_l2_l3   1973 non-null   object
 8   left_neural_foraminal_narrowing_l3_l4   1973 non-null   object
 9   left_neural_foraminal_narrowing_l4_l5   1973 non-null   object
 10  left_neural_foraminal_narrowing_l5_s1   1973 non-null   object
 11  righ

In [7]:
df_train_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48692 entries, 0 to 48691
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   study_id         48692 non-null  int64  
 1   series_id        48692 non-null  int64  
 2   instance_number  48692 non-null  int64  
 3   condition        48692 non-null  object 
 4   level            48692 non-null  object 
 5   x                48692 non-null  float64
 6   y                48692 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 2.6+ MB


In [8]:
# look at categories
for f in ['instance_number','condition','level']:
    print(df_train_label[f].value_counts())
    print('-'*50);print();

instance_number
8      3615
11     3517
10     3396
5      3288
12     3180
       ... 
118       1
96        1
73        1
83        1
117       1
Name: count, Length: 101, dtype: int64
--------------------------------------------------

condition
Left Neural Foraminal Narrowing     9860
Right Neural Foraminal Narrowing    9859
Spinal Canal Stenosis               9753
Right Subarticular Stenosis         9612
Left Subarticular Stenosis          9608
Name: count, dtype: int64
--------------------------------------------------

level
L3/L4    9858
L4/L5    9858
L5/S1    9845
L2/L3    9661
L1/L2    9470
Name: count, dtype: int64
--------------------------------------------------



In [9]:
# join first two tables
df_train_step_1 = pd.merge(left=df_train_label, right=df_train_main, how='left', on='study_id').reset_index(drop=True)
df_train_step_1.head()

,study_id,series_id,instance_number,condition,level,x,y,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild


In [10]:
# join with third table
df_train = pd.merge(left=df_train_step_1, right=df_train_desc, how='left', on=['study_id', 'series_id']).reset_index(drop=True)
df_train.head()

,study_id,series_id,instance_number,condition,level,x,y,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,...,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1,series_description
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T2/STIR
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T2/STIR
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T2/STIR
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T2/STIR
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T2/STIR


In [11]:
# convert study_id's to categorical
df_train.study_id = df_train.study_id.astype('category')
df_train.series_id = df_train.series_id.astype('category')
df_train.head()

,study_id,series_id,instance_number,condition,level,x,y,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,...,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1,series_description
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T2/STIR
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T2/STIR
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T2/STIR
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T2/STIR
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Sagittal T2/STIR


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48692 entries, 0 to 48691
Data columns (total 33 columns):
 #   Column                                  Non-Null Count  Dtype   
---  ------                                  --------------  -----   
 0   study_id                                48692 non-null  category
 1   series_id                               48692 non-null  category
 2   instance_number                         48692 non-null  int64   
 3   condition                               48692 non-null  object  
 4   level                                   48692 non-null  object  
 5   x                                       48692 non-null  float64 
 6   y                                       48692 non-null  float64 
 7   spinal_canal_stenosis_l1_l2             48692 non-null  object  
 8   spinal_canal_stenosis_l2_l3             48692 non-null  object  
 9   spinal_canal_stenosis_l3_l4             48692 non-null  object  
 10  spinal_canal_stenosis_l4_l5             48692 

In [13]:
class ParticipantVisibleError(Exception):
    pass


In [14]:
# 문자열에서 'spinal', 'foraminal', 'subarticular' 셋중 하나가 없으면 valueerror 반환
def get_condition(full_location: str) -> str:
    # Given an input like spinal_canal_stenosis_l1_l2 extracts 'spinal'
    for injury_condition in ['spinal', 'foraminal', 'subarticular']:
        if injury_condition in full_location:
            return injury_condition
    raise ValueError(f'condition not found in {full_location}')

In [15]:
'''
의사 코드
1. 각 의학적 상태에 대한 샘플 가중 로그 손실을 계산합니다
2. 새로운 any_sever 레이블을 유도합니다.
3. 새로운 any_sever 레이블에 대한 샘플 가중 로그 손실을 계산합니다.
4. 모든 레이블 그룹 로그 손실의 평균을 각 그룹의 열 수에 대해 정규화된 최종 점수로 반환합니다.
    이것은 다른 두 가지 조건의 절반에 불과한 척추 협착증의 영향을 완화시킵니다.
'''

'\n의사 코드\n1. 각 의학적 상태에 대한 샘플 가중 로그 손실을 계산합니다\n2. 새로운 any_sever 레이블을 유도합니다.\n3. 새로운 any_sever 레이블에 대한 샘플 가중 로그 손실을 계산합니다.\n4. 모든 레이블 그룹 로그 손실의 평균을 각 그룹의 열 수에 대해 정규화된 최종 점수로 반환합니다.\n    이것은 다른 두 가지 조건의 절반에 불과한 척추 협착증의 영향을 완화시킵니다.\n'

In [16]:
def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        row_id_column_name: str,
        any_severe_scalar: float
    ) -> float:
    
    '''
        의사 코드
        1. 각 의학적 상태에 대한 샘플 가중 로그 손실을 계산합니다
        2. 새로운 any_sever 레이블을 유도합니다.
        3. 새로운 any_sever 레이블에 대한 샘플 가중 로그 손실을 계산합니다.
        4. 모든 레이블 그룹 로그 손실의 평균을 각 그룹의 열 수에 대해 정규화된 최종 점수로 반환합니다.
            이것은 다른 두 가지 조건의 절반에 불과한 척추 협착증의 영향을 완화시킵니다.
    '''

    target_levels = ['normal_mild', 'moderate', 'severe']

    # Run basic QC checks on the inputs
    if not pandas.api.types.is_numeric_dtype(submission[target_levels].values):
        raise ParticipantVisibleError('All submission values must be numeric')

    if not np.isfinite(submission[target_levels].values).all():
        raise ParticipantVisibleError('All submission values must be finite')

    if solution[target_levels].min().min() < 0:
        raise ParticipantVisibleError('All labels must be at least zero')
    if submission[target_levels].min().min() < 0:
        raise ParticipantVisibleError('All predictions must be at least zero')

    solution['study_id']  = solution['row_id'].apply(lambda x: x.split('_')[0])
    solution['location']  = solution['row_id'].apply(lambda x: '_'.join(x.split('_')[1:]))
    solution['condition'] = solution['row_id'].apply(get_condition)

    del solution[row_id_column_name]
    del submission[row_id_column_name]
    assert sorted(submission.columns) == sorted(target_levels)

    submission['study_id']  = solution['study_id']
    submission['location']  = solution['location']
    submission['condition'] = solution['condition']

    condition_losses  = []
    condition_weights = []
    for condition in ['spinal', 'foraminal', 'subarticular']:
        condition_indices = solution.loc[solution['condition'] == condition].index.values
        condition_loss = sklearn.metrics.log_loss(
            y_true=solution.loc[condition_indices, target_levels].values,
            y_pred=submission.loc[condition_indices, target_levels].values,
            sample_weight=solution.loc[condition_indices, 'sample_weight'].values
        )
        condition_losses.append(condition_loss)
        condition_weights.append(1 / solution.loc[condition_indices, 'location'].nunique())

        
    any_severe_spinal_labels      = pd.Series(solution.loc[solution    ['condition'] == 'spinal'].groupby('study_id')['severe'].max())
    any_severe_spinal_weights     = pd.Series(solution.loc[solution    ['condition'] == 'spinal'].groupby('study_id')['sample_weight'].max())
    any_severe_spinal_predictions = pd.Series(submission.loc[submission['condition'] == 'spinal'].groupby('study_id')['severe'].max())
    any_severe_spinal_loss = sklearn.metrics.log_loss(
        y_true=any_severe_spinal_labels,
        y_pred=any_severe_spinal_predictions,
        sample_weight=any_severe_spinal_weights
    )
    condition_losses.append(any_severe_spinal_loss)
    condition_weights.append(any_severe_scalar)
    return np.average(condition_losses, weights=condition_weights)

In [64]:
from sklearn.preprocessing import LabelEncoder

In [65]:
test_series = pd.read_csv(path + 'test_series_descriptions.csv')
df_train = pd.read_csv(path + 'train.csv')

In [66]:
df_train

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,4282019580,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate
1971,4283570761,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1972,4284048608,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild
1973,4287160193,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Severe,Moderate,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Moderate,Normal/Mild


In [67]:
df_train_melted = df_train.melt(id_vars=['study_id'], var_name='condition_level', value_name='severity')
df_train_melted

,study_id,condition_level,severity
0,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild
1,4646740,spinal_canal_stenosis_l1_l2,Normal/Mild
2,7143189,spinal_canal_stenosis_l1_l2,Normal/Mild
3,8785691,spinal_canal_stenosis_l1_l2,Normal/Mild
4,10728036,spinal_canal_stenosis_l1_l2,Normal/Mild
...,...,...,...
49370,4282019580,right_subarticular_stenosis_l5_s1,Moderate
49371,4283570761,right_subarticular_stenosis_l5_s1,Normal/Mild
49372,4284048608,right_subarticular_stenosis_l5_s1,Normal/Mild
49373,4287160193,right_subarticular_stenosis_l5_s1,Normal/Mild


In [82]:
df_train_melted['level']=df_train_melted['condition_level'].apply(lambda x: '_'.join(x.split('_')[-2:]))

In [83]:
df_train_melted['condition']=df_train_melted['condition_level'].str.rsplit('_', n=2, expand=True).iloc[:,0]
df_train_melted

,study_id,condition_level,severity,level,condition,severity_encoded
0,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
1,4646740,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
2,7143189,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
3,8785691,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
4,10728036,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
...,...,...,...,...,...,...
49370,4282019580,right_subarticular_stenosis_l5_s1,Moderate,l5_s1,right_subarticular_stenosis,0
49371,4283570761,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis,1
49372,4284048608,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis,1
49373,4287160193,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis,1


In [84]:
le_severity = LabelEncoder()

In [85]:
# labelencoder ['Moderate', 'Normal/Mild', 'Severe', nan]
df_train_melted['severity_encoded'] = le_severity.fit_transform(df_train_melted['severity'])
df_train_melted

,study_id,condition_level,severity,level,condition,severity_encoded
0,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
1,4646740,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
2,7143189,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
3,8785691,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
4,10728036,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
...,...,...,...,...,...,...
49370,4282019580,right_subarticular_stenosis_l5_s1,Moderate,l5_s1,right_subarticular_stenosis,0
49371,4283570761,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis,1
49372,4284048608,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis,1
49373,4287160193,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis,1


In [86]:
le_severity.classes_

array(['Moderate', 'Normal/Mild', 'Severe', nan], dtype=object)

In [87]:
X_train = df_train_melted[['study_id', 'condition', 'level']]
X_train

,study_id,condition,level
0,4003253,spinal_canal_stenosis,l1_l2
1,4646740,spinal_canal_stenosis,l1_l2
2,7143189,spinal_canal_stenosis,l1_l2
3,8785691,spinal_canal_stenosis,l1_l2
4,10728036,spinal_canal_stenosis,l1_l2
...,...,...,...
49370,4282019580,right_subarticular_stenosis,l5_s1
49371,4283570761,right_subarticular_stenosis,l5_s1
49372,4284048608,right_subarticular_stenosis,l5_s1
49373,4287160193,right_subarticular_stenosis,l5_s1


In [88]:
y_train = df_train_melted['severity_encoded']
y_train

0        1
1        1
2        1
3        1
4        1
        ..
49370    0
49371    1
49372    1
49373    1
49374    1
Name: severity_encoded, Length: 49375, dtype: int32

In [89]:
X_train = pd.get_dummies(X_train, columns=['condition', 'level'])
X_train

,study_id,condition_left_neural_foraminal_narrowing,condition_left_subarticular_stenosis,condition_right_neural_foraminal_narrowing,condition_right_subarticular_stenosis,condition_spinal_canal_stenosis,level_l1_l2,level_l2_l3,level_l3_l4,level_l4_l5,level_l5_s1
0,4003253,False,False,False,False,True,True,False,False,False,False
1,4646740,False,False,False,False,True,True,False,False,False,False
2,7143189,False,False,False,False,True,True,False,False,False,False
3,8785691,False,False,False,False,True,True,False,False,False,False
4,10728036,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
49370,4282019580,False,False,False,True,False,False,False,False,False,True
49371,4283570761,False,False,False,True,False,False,False,False,False,True
49372,4284048608,False,False,False,True,False,False,False,False,False,True
49373,4287160193,False,False,False,True,False,False,False,False,False,True


In [100]:
test_rows = []
for _, row in test_series.iterrows():
    for condition in ['left_neural_foraminal_narrowing', 'right_neural_foraminal_narrowing', 'left_subarticular_stenosis', 'right_subarticular_stenosis', 'spinal_canal_stenosis']:
        for level in ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']:
            test_rows.append({
                'study_id': row['study_id'],
                'condition': condition,
                'level': level
            })

X_test = pd.DataFrame(test_rows)
X_test

,study_id,condition,level
0,44036939,left_neural_foraminal_narrowing,l1_l2
1,44036939,left_neural_foraminal_narrowing,l2_l3
2,44036939,left_neural_foraminal_narrowing,l3_l4
3,44036939,left_neural_foraminal_narrowing,l4_l5
4,44036939,left_neural_foraminal_narrowing,l5_s1
...,...,...,...
70,44036939,spinal_canal_stenosis,l1_l2
71,44036939,spinal_canal_stenosis,l2_l3
72,44036939,spinal_canal_stenosis,l3_l4
73,44036939,spinal_canal_stenosis,l4_l5


In [101]:
X_test = pd.get_dummies(X_test, columns=['condition', 'level'])
X_test

,study_id,condition_left_neural_foraminal_narrowing,condition_left_subarticular_stenosis,condition_right_neural_foraminal_narrowing,condition_right_subarticular_stenosis,condition_spinal_canal_stenosis,level_l1_l2,level_l2_l3,level_l3_l4,level_l4_l5,level_l5_s1
0,44036939,True,False,False,False,False,True,False,False,False,False
1,44036939,True,False,False,False,False,False,True,False,False,False
2,44036939,True,False,False,False,False,False,False,True,False,False
3,44036939,True,False,False,False,False,False,False,False,True,False
4,44036939,True,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
70,44036939,False,False,False,False,True,True,False,False,False,False
71,44036939,False,False,False,False,True,False,True,False,False,False
72,44036939,False,False,False,False,True,False,False,True,False,False
73,44036939,False,False,False,False,True,False,False,False,True,False


In [103]:
# Pandas DataFrame의 열 정렬과 결측값 처리
# X_test DataFrame이 X_train DataFrame의 열 순서와 열 이름에 맞도록 재정렬하고, 
# X_train에 없는 열이 X_test에 있는 경우 그 열의 값을 0으로 채움.
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
X_test

,study_id,condition_left_neural_foraminal_narrowing,condition_left_subarticular_stenosis,condition_right_neural_foraminal_narrowing,condition_right_subarticular_stenosis,condition_spinal_canal_stenosis,level_l1_l2,level_l2_l3,level_l3_l4,level_l4_l5,level_l5_s1
0,44036939,True,False,False,False,False,True,False,False,False,False
1,44036939,True,False,False,False,False,False,True,False,False,False
2,44036939,True,False,False,False,False,False,False,True,False,False
3,44036939,True,False,False,False,False,False,False,False,True,False
4,44036939,True,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
70,44036939,False,False,False,False,True,True,False,False,False,False
71,44036939,False,False,False,False,True,False,True,False,False,False
72,44036939,False,False,False,False,True,False,False,True,False,False
73,44036939,False,False,False,False,True,False,False,False,True,False


In [104]:
X_train

,study_id,condition_left_neural_foraminal_narrowing,condition_left_subarticular_stenosis,condition_right_neural_foraminal_narrowing,condition_right_subarticular_stenosis,condition_spinal_canal_stenosis,level_l1_l2,level_l2_l3,level_l3_l4,level_l4_l5,level_l5_s1
0,4003253,False,False,False,False,True,True,False,False,False,False
1,4646740,False,False,False,False,True,True,False,False,False,False
2,7143189,False,False,False,False,True,True,False,False,False,False
3,8785691,False,False,False,False,True,True,False,False,False,False
4,10728036,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
49370,4282019580,False,False,False,True,False,False,False,False,False,True
49371,4283570761,False,False,False,True,False,False,False,False,False,True
49372,4284048608,False,False,False,True,False,False,False,False,False,True
49373,4287160193,False,False,False,True,False,False,False,False,False,True


In [127]:
y_train

0        1
1        1
2        1
3        1
4        1
        ..
49370    0
49371    1
49372    1
49373    1
49374    1
Name: severity_encoded, Length: 49375, dtype: int32

In [106]:
from xgboost import XGBClassifier

model = XGBClassifier(random_state=1)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [108]:
predictions_proba = model.predict_proba(X_test)
predictions_df = pd.DataFrame(predictions_proba, columns=le_severity.classes_)
predictions_df['study_id'] = X_test['study_id'].values
predictions_df['condition_level'] = X_test.index.map(lambda idx: f"{test_rows[idx]['condition']}_{test_rows[idx]['level']}")
predictions_df['row_id'] = predictions_df['study_id'].astype(str) + '_' + predictions_df['condition_level']
predictions_df

,Moderate,Normal/Mild,Severe,NaN,study_id,condition_level,row_id
0,0.013609,0.985949,0.000431,0.000011,44036939,left_neural_foraminal_narrowing_l1_l2,44036939_left_neural_foraminal_narrowing_l1_l2
1,0.045299,0.954271,0.000406,0.000024,44036939,left_neural_foraminal_narrowing_l2_l3,44036939_left_neural_foraminal_narrowing_l2_l3
2,0.144020,0.852448,0.003458,0.000073,44036939,left_neural_foraminal_narrowing_l3_l4,44036939_left_neural_foraminal_narrowing_l3_l4
3,0.477195,0.506100,0.016665,0.000039,44036939,left_neural_foraminal_narrowing_l4_l5,44036939_left_neural_foraminal_narrowing_l4_l5
4,0.338026,0.640009,0.021915,0.000051,44036939,left_neural_foraminal_narrowing_l5_s1,44036939_left_neural_foraminal_narrowing_l5_s1
...,...,...,...,...,...,...,...
70,0.029824,0.961291,0.008848,0.000036,44036939,spinal_canal_stenosis_l1_l2,44036939_spinal_canal_stenosis_l1_l2
71,0.195061,0.772116,0.032750,0.000073,44036939,spinal_canal_stenosis_l2_l3,44036939_spinal_canal_stenosis_l2_l3
72,0.358443,0.593499,0.047937,0.000121,44036939,spinal_canal_stenosis_l3_l4,44036939_spinal_canal_stenosis_l3_l4
73,0.152337,0.562395,0.285223,0.000045,44036939,spinal_canal_stenosis_l4_l5,44036939_spinal_canal_stenosis_l4_l5


In [126]:
predictions_proba

array([[1.36087164e-02, 9.85948563e-01, 4.31421096e-04, 1.12975904e-05],
       [4.52990234e-02, 9.54271019e-01, 4.05980711e-04, 2.40307018e-05],
       [1.44020244e-01, 8.52448463e-01, 3.45806917e-03, 7.32395129e-05],
       [4.77195472e-01, 5.06100476e-01, 1.66651998e-02, 3.88428307e-05],
       [3.38026077e-01, 6.40008867e-01, 2.19145548e-02, 5.05005919e-05],
       [1.47935972e-02, 9.65948403e-01, 1.90391578e-02, 2.18828543e-04],
       [7.78153464e-02, 9.19394195e-01, 2.11066566e-03, 6.79756980e-04],
       [2.25032255e-01, 7.58974731e-01, 1.55712338e-02, 4.21727804e-04],
       [4.02684182e-01, 5.64624071e-01, 3.24678160e-02, 2.23923693e-04],
       [3.27538073e-01, 6.00912809e-01, 7.12433234e-02, 3.05781199e-04],
       [6.16584532e-02, 9.11120415e-01, 1.95757877e-02, 7.64538441e-03],
       [1.39879957e-01, 8.35526884e-01, 1.74624361e-02, 7.13074300e-03],
       [3.18394631e-01, 6.07792079e-01, 7.23108128e-02, 1.50245673e-03],
       [3.52497965e-01, 4.24173623e-01, 2.23112985e

In [112]:
df_sub1 = pd.read_csv(path + 'sample_submission.csv')
df_sub1

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.333333,0.333333,0.333333
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.333333,0.333333,0.333333
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.333333,0.333333,0.333333
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.333333,0.333333,0.333333
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.333333,0.333333,0.333333
5,44036939_left_subarticular_stenosis_l1_l2,0.333333,0.333333,0.333333
6,44036939_left_subarticular_stenosis_l2_l3,0.333333,0.333333,0.333333
7,44036939_left_subarticular_stenosis_l3_l4,0.333333,0.333333,0.333333
8,44036939_left_subarticular_stenosis_l4_l5,0.333333,0.333333,0.333333
9,44036939_left_subarticular_stenosis_l5_s1,0.333333,0.333333,0.333333


In [113]:
predictions_df

,Moderate,Normal/Mild,Severe,NaN,study_id,condition_level,row_id
0,0.013609,0.985949,0.000431,0.000011,44036939,left_neural_foraminal_narrowing_l1_l2,44036939_left_neural_foraminal_narrowing_l1_l2
1,0.045299,0.954271,0.000406,0.000024,44036939,left_neural_foraminal_narrowing_l2_l3,44036939_left_neural_foraminal_narrowing_l2_l3
2,0.144020,0.852448,0.003458,0.000073,44036939,left_neural_foraminal_narrowing_l3_l4,44036939_left_neural_foraminal_narrowing_l3_l4
3,0.477195,0.506100,0.016665,0.000039,44036939,left_neural_foraminal_narrowing_l4_l5,44036939_left_neural_foraminal_narrowing_l4_l5
4,0.338026,0.640009,0.021915,0.000051,44036939,left_neural_foraminal_narrowing_l5_s1,44036939_left_neural_foraminal_narrowing_l5_s1
...,...,...,...,...,...,...,...
70,0.029824,0.961291,0.008848,0.000036,44036939,spinal_canal_stenosis_l1_l2,44036939_spinal_canal_stenosis_l1_l2
71,0.195061,0.772116,0.032750,0.000073,44036939,spinal_canal_stenosis_l2_l3,44036939_spinal_canal_stenosis_l2_l3
72,0.358443,0.593499,0.047937,0.000121,44036939,spinal_canal_stenosis_l3_l4,44036939_spinal_canal_stenosis_l3_l4
73,0.152337,0.562395,0.285223,0.000045,44036939,spinal_canal_stenosis_l4_l5,44036939_spinal_canal_stenosis_l4_l5


In [115]:
predictions_df['Normal/Mild']

0     0.985949
1     0.954271
2     0.852448
3     0.506100
4     0.640009
        ...   
70    0.961291
71    0.772116
72    0.593499
73    0.562395
74    0.987298
Name: Normal/Mild, Length: 75, dtype: float32

In [119]:
normal_mild_value = predictions_df['Normal/Mild']
moderate_value = predictions_df['Moderate']
severe_value = predictions_df['Severe']


df_sub['normal_mild'] = normal_mild_value/2
df_sub['moderate'] =moderate_value*2
df_sub['severe'] = 1-(normal_mild_value/2 +moderate_value*2)- severe_value

In [123]:
df_sub['normal_mild'] = normal_mild_value
df_sub['moderate']    = moderate_value
df_sub['severe']      = severe_value
df_sub

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.985949,0.013609,0.000431
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.954271,0.045299,0.000406
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.852448,0.144020,0.003458
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.506100,0.477195,0.016665
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.640009,0.338026,0.021915
5,44036939_left_subarticular_stenosis_l1_l2,0.965948,0.014794,0.019039
6,44036939_left_subarticular_stenosis_l2_l3,0.919394,0.077815,0.002111
7,44036939_left_subarticular_stenosis_l3_l4,0.758975,0.225032,0.015571
8,44036939_left_subarticular_stenosis_l4_l5,0.564624,0.402684,0.032468
9,44036939_left_subarticular_stenosis_l5_s1,0.600913,0.327538,0.071243


In [122]:
df_sub.to_csv('submission.csv', index=False)

In [125]:
'''
kaggle 점수 : 1.30
'''

'\nkaggle 점수 : 1.30\n'